In [5]:
import os
import re
import unicodedata

In [4]:
from lingtrain_aligner import preprocessor, splitter, aligner, resolver, helper, reader, vis_helper

ModuleNotFoundError: No module named 'lingtrain_aligner'

In [101]:
aligner.align_texts.__doc__

In [102]:
with open('JP_eng.txt', 'r') as f:
    data = f.read()

with open('JP_eng.txt', 'w') as f:
    f.write(re.sub(r'”|“', '', data))

In [103]:
source = open('JP_de.txt','r',encoding='utf-8')
source_line=source.readlines()
target = open('JP_eng.txt','r',encoding='utf-8')
target_line=target.readlines()

In [104]:
source_prepared = preprocessor.mark_paragraphs(source_line)
target_prepared = preprocessor.mark_paragraphs(target_line)
splitted_from = splitter.split_by_sentences_wrapper(source_prepared, 'de', leave_marks=True)
splitted_to = splitter.split_by_sentences_wrapper(target_prepared, 'eng', leave_marks=True)

In [105]:
print(len(splitted_to))
print(len(splitted_from))

5805
5898


In [108]:
#создаем базу данных

db_path='result_db.db'
if os.path.isfile(db_path):
    os.unlink(db_path)

aligner.fill_db(db_path,'de', 'eng', splitted_from, splitted_to)

In [ ]:
#models = ["sentence_transformer_multilingual", "sentence_transformer_multilingual_labse"]
model_name = "sentence_transformer_multilingual"
batch_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]

db=aligner.align_db(db_path, \
                model_name, \
                batch_size=100, \
                window=40, \
                batch_ids=batch_ids, \
                save_pic=False,
                embed_batch_size=10, \
                normalize_embeddings=True, \
                show_progress_bar=True
                )

In [ ]:
vis_helper.visualize_alignment_by_db(db_path, output_path="alignment_vis.png", \
                                    lang_name_from=lang_from, \
                                    lang_name_to=lang_to, \
                                    batch_size=5800, \
                                    size=(800,800), \
                                    plt_show=True)

In [110]:
conflicts_to_solve, rest = resolver.get_all_conflicts(db_path, min_chain_length=2, max_conflicts_len=6, batch_id=-1)

resolver.get_statistics(conflicts_to_solve)
resolver.get_statistics(rest)

conflicts to solve: 366
total conflicts: 372
('2:1', 113)
('2:3', 99)
('3:2', 59)
('3:3', 34)
('4:2', 11)
('3:5', 8)
('3:1', 8)
('4:4', 6)
('3:4', 6)
('4:3', 3)
('2:4', 3)
('6:3', 2)
('5:3', 2)
('5:5', 2)
('4:6', 2)
('4:5', 2)
('6:6', 2)
('5:2', 1)
('5:4', 1)
('3:6', 1)
('6:4', 1)
('7:3', 1)
('5:8', 1)
('6:8', 1)
('4:-12', 1)
('3:19', 1)
('9:8', 1)


In [111]:
resolver.show_conflict(db_path, conflicts_to_solve[10])

173 Alle haben gesagt, wie gut sie aussieht, und Mr. Bingley fand sie sehr schön und hat zweimal mit ihr getanzt – denk dir nur, mein Lieber, er hat tatsächlich zweimal mit ihr getanzt; und sie war das einzige Mädchen im Saal, das er zweimal aufgefordert hat.
174 Zuallererst hat er Miss Lucas aufgefordert.
175 Ich war so ärgerlich, als er mit ihr zum Tanz antrat; aber er hat sie überhaupt nicht bewundert – wahrhaftig, niemand kann das, nicht wahr –; aber er schien ganz begeistert von Jane, als er sie beim Tanz sah.


173 Everybody said how well she looked; and Mr. Bingley thought her quite beautiful, and danced with her twice!
174 Only think of that , my dear; he actually danced with her twice! and she was the only creature in the room that he asked a second time.
175 First of all, he asked Miss Lucas.
176 I was so vexed to see him stand up with her!
177 But, however, he did not admire her at all; indeed, nobody can, you know; and he seemed quite struck with Jane as she was going down 

In [ ]:
steps = 3
batch_id = -1  #all batches

for i in range(steps):
    conflicts, rest = resolver.get_all_conflicts(db_path, min_chain_length=2+i, max_conflicts_len=6*(i+1), batch_id=batch_id)

    resolver.resolve_all_conflicts(db_path, conflicts, model_name, show_logs=False)

    vis_helper.visualize_alignment_by_db(db_path, output_path="img_test1.png", batch_size=5800, size=(800,800), plt_show=True)

    if len(rest) == 0:
        break

In [113]:
from lingtrain_aligner import reader

paragraphs_from, paragraphs_to, meta = reader.get_paragraphs(db_path, direction="from")
reader.create_book(paragraphs_from, 
                    paragraphs_to,
                    meta,
                    output_path = "prpr.html",
                    template="pastel_fill",
                    )

In [114]:
from bs4 import BeautifulSoup

with open("lingtrain.html", "r", encoding="utf-8") as f:
    html_content = f.read()        
    soup = BeautifulSoup(html_content, 'html.parser')
    new_img_tag = soup.new_tag('img', src="pp.jpg")
    style_tag = soup.new_tag('style')
    style_content = """
        body {
            margin: 0;
            display: flex;
            flex-direction: column; 
            align-items: center;
        }
        img {
            max-width: 100%; 
            height: auto; 
            margin-top: 20px;
        }
    """
    style_tag.string = style_content
    target_element = soup.select_one("head")
    target_element.append(style_tag)
    target_body = soup.select_one("body")
    target_body.insert(0, new_img_tag) 

with open("lingtrain.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify(formatter="html"))